The following notebook is for calculating the angular resolution and stellar radius of a star based off the parallax measurement from either Gaia or Hipparcos, as well as effective temperature and bolometric flux. It will also calculate the errors in each of these calculated values. Descriptions precede each section.

---

The first section loads relevant packages

In [1]:
import numpy
from astropy import constants as c
from astropy import units as u
from astropy.coordinates import SkyCoord

The first section is for making corrections to the Gaia parallax, in line with the paper by Stassun and Torres indicating a systematic bias in the parallax measurements from the Gaia DR1 data.  
doi:10.3847/2041-8205/831/1/L6

The coordinate string contains the RA and Dec of the star, and needs to be in hour angle and degrees, respectively.  
pH is the Hipparcos parallax, and pG is the Gaia parallax. The units are required, and so switch out the numbers, for the star in question, and the appropriate parallax measurements.

In [2]:
CoordinateString='19 38 38.73445 +31 13 09.211'
pH=8.73*u.mas
pG=7.16*u.mas

The following block will print the parallax (mas) and distance (pc) for four cases provided that all the necessary parameters were included above  
* Hipparcos parallax
* Gaia parallax (uncorrected)
* Gaia parallax (corrected using the median systematic bias)  *recommended*
* Gaia parallax (corrected using bias as a function of ecliptic latitude)

In [3]:
if pH:
    print "Hipparcos:", pH, pH.to(u.parsec, equivalencies=u.parallax())
          
if pG:
    print "Gaia:", pG, pG.to(u.parsec, equivalencies=u.parallax())
    del_pi=-0.25*u.mas
    corpG=(pG)-del_pi
    print "Gaia cor:", corpG, corpG.to(u.parsec, equivalencies=u.parallax())
    if CoordinateString:
        eq_pos = SkyCoord(CoordinateString, unit=(u.hourangle, u.deg), frame='icrs')
        ecl_degree=eq_pos.barycentrictrueecliptic.lat.value
        #print "Ecliptic latitude", ecl_degree
        del_pi=(-0.22-0.003*ecl_degree)*u.mas
        corpG=(pG)-del_pi
        print "Gaia Lat cor:", corpG, corpG.to(u.parsec, equivalencies=u.parallax())

Hipparcos: 8.73 mas 114.547537228 pc
Gaia: 7.16 mas 139.664804469 pc
Gaia cor: 7.41 mas 134.952766532 pc
Gaia Lat cor: 7.53508034903 mas 132.712586154 pc


Input the values and errors on the stellar effective temperature (Teff), bolometric flux (Fbol), and parallax in the next block

In [6]:
Teff=8800*u.K
Terr=500*u.K
Fbol=2.46* 10**-8*u.erg/u.s/u.cm**2
Fbolerr=0.27* 10**-8*u.erg/u.s/u.cm**2
parallax=7.41*u.mas
parallaxerr=0.39*u.mas

The final section will calculate the stellar angular diameter (mas), stellar distance (pc), and stellar radius (Solar radii) based off of the values entered above.

In [7]:
angR= ((Fbol/(c.sigma_sb*Teff**4))**0.5).decompose()
distance= parallax.to(u.parsec, equivalencies=u.parallax())
radius=(angR*distance).to(u.solRad)
print "Stellar angular diameter:", (angR*u.rad).to(u.mas)
print "Stellar distance:", distance
print "Stellar radius:", radius

A=(0.5*Fbolerr/Fbol)**2 #F_bol
B=(2*Terr/Teff)**2 #Teff
C=(parallaxerr/parallax)**2 #parallax
print "Angular diameter error:", ((A+B)**0.5)*(angR*u.rad).to(u.mas)
print "Distance error:", ((C)**0.5)*distance
print "Radius error:", ((A+B+C)**0.5)*radius

Stellar angular diameter: 0.0554780935373 mas
Stellar distance: 134.952766532 pc
Stellar radius: 1.61037345353 solRad
Angular diameter error: 0.0070009800583 mas
Distance error: 7.10277718588 pc
Radius error: 0.220185264499 solRad
